To Do
1. Run the Python Script
2. Run RDP Deviation Script in Excel > Automate

In [ ]:
import pandas as pd
import openpyxl
import os

# Source file path and header file path
source_file_path = r"D:\23. RDP Survey Script\0. UPDATE PROGRESS EXPLORATION-SURVEY 2024_(13-07-2024).xlsx"
header_file_path = r"D:\23. RDP Survey Script\Script & Header\RDP Survey Header.xlsx"

# Open source file with openpyxl
workbook = openpyxl.load_workbook(source_file_path)

# Function to unmerge cells
def unmerge_header(sheet):
    merged_cells = list(sheet.merged_cells)
    for merged_cell in merged_cells:
        sheet.unmerge_cells(str(merged_cell))

# Unmerge header in each sheet
for sheet_name in workbook.sheetnames[:13]:  # Only take the first 13 sheets
    sheet = workbook[sheet_name]
    unmerge_header(sheet)

# Save changes to a temporary file
temp_file_path = source_file_path
workbook.save(temp_file_path)

# Read header from header file
header_df = pd.read_excel(header_file_path)
header = header_df.columns.tolist()[:29]  # Only take the first 29 columns of the header

# Read all sheets from the temporary file
sheets = pd.read_excel(temp_file_path, sheet_name=None)

# Combine data from the first 13 sheets
combined_data = pd.DataFrame()

for sheet_name in list(sheets.keys())[:13]:  # Only take the first 13 sheets
    sheet_df = sheets[sheet_name]
    # Remove rows 1 and 2 which contain the sheet header
    data = sheet_df.iloc[2:, :29]  # Only take the first 29 columns
    # Keep only rows where column F is not null
    data = data[data.iloc[:, 5].notnull()]
    # Add data to the combined data
    combined_data = pd.concat([combined_data, data], ignore_index=True)

# Add header taken from header file
combined_data.columns = header

# Determine output file path
output_file_path = os.path.join(os.path.dirname(source_file_path), 
                                os.path.basename(source_file_path).replace('.xlsx', '.csv'))

# Save combined data to a CSV file
combined_data.to_csv(output_file_path, index=False)

# Remove temporary file
os.remove(temp_file_path)

# Remove spaces in the 6th column (column F) from the saved CSV file
combined_data = pd.read_csv(output_file_path)

# Use numerical index 5 to refer to the 6th column
combined_data.iloc[:, 5] = combined_data.iloc[:, 5].str.strip()  # Remove spaces around text

# Save back to CSV file
combined_data.to_csv(output_file_path, index=False)

print(f"CSV file has been saved to: {output_file_path}")
